In [116]:
from dataclasses import dataclass

@dataclass
class ServiceNetwork:
    total_nmo_stake: int = 0
    total_lp_supply: int = 0
    
    def stake(self, nmo):
        if self.total_nmo_stake == 0:
            # this is the only staker in the network
            self.total_nmo_stake = nmo
            self.total_lp_supply = 1.0
            return self.total_lp_supply
        
        lp_tokens = nmo / self.total_nmo_stake * self.total_lp_supply
        self.total_nmo_stake += nmo
        self.total_lp_supply += lp_tokens
        return lp_tokens

    def unstake(self, lp_tokens):
        nmo = self.query_stake(lp_tokens)
        self.total_nmo_stake -= nmo
        self.total_lp_supply -= lp_tokens
        return nmo

    def query_stake(self, lp_tokens):
        nmo = lp_tokens / self.total_lp_supply * self.total_nmo_stake    
        return nmo
    
    def reward(self, r_nmo):
        self.total_nmo_stake += r_nmo

def print_lp_positions(sn, lpers):
    print(f"{"actor":^7}|{"lp":^5}|{"nmo":^7}")
    print("-------+-----+-------")
    for actor, lp in lpers.items():
        print(f"{actor:7}|{lp:5.2f}|{sn.query_stake(lp):7.2f}")

    print("-------+-----+-------")
    print("{actor:7}|{lp:5.2f}|{nmo:7.2f}".format(actor="total", lp=sum(lpers.values()), nmo=sum(sn.query_stake(lp) for lp in lpers.values())))

In [119]:
sn = ServiceNetwork()

stakers = {"alice": 10, "bob": 10, "charlie": 30}

lpers = {actor: sn.stake(nmo) for actor, nmo in stakers.items()}

print_lp_positions(sn, lpers)

print("")
print("network is rewarded by 5 NMO")
sn.reward(5)
print("")

print_lp_positions(sn, lpers)

print("")
print("new stakers after network rewards get fewer LP tokens")
print("")
stakers["dan"] = 10
lpers["dan"] = sn.stake(stakers["dan"])
print_lp_positions(sn, lpers)


 actor | lp  |  nmo  
-------+-----+-------
alice  | 1.00|  10.00
bob    | 1.00|  10.00
charlie| 3.00|  30.00
-------+-----+-------
total  | 5.00|  50.00

network is rewarded by 5 NMO

 actor | lp  |  nmo  
-------+-----+-------
alice  | 1.00|  11.00
bob    | 1.00|  11.00
charlie| 3.00|  33.00
-------+-----+-------
total  | 5.00|  55.00

new stakers after network rewards get fewer LP tokens

 actor | lp  |  nmo  
-------+-----+-------
alice  | 1.00|  11.00
bob    | 1.00|  11.00
charlie| 3.00|  33.00
dan    | 0.91|  10.00
-------+-----+-------
total  | 5.91|  65.00
